In [1]:
import sal_ssim
import sal_ssim_fullframe
import piq

import numpy as np
from pathlib import Path
import os
from time import time
import pickle as pkl
from utils import *

import logging


In [2]:
DATASET = Path('/home/a_mos/2019')
OUTPUT = Path(os.getcwd()) / 'results_17_08'

In [3]:
PKL_ARRAYS = OUTPUT / 'pkl_arrays'
RESULTS = OUTPUT / 'raw.csv'
ERRORS =  OUTPUT / 'errors.txt'
LOG_FILE = OUTPUT / 'log.txt'

HEADERS = 'Name,Reference,Distorted,Metric,Metric_val,Mask,Time'.split(',')

if not os.path.exists(OUTPUT):
    os.mkdir(str(OUTPUT))
if not os.path.exists(PKL_ARRAYS):
    os.mkdir(str(PKL_ARRAYS))
    
log = get_logger(LOG_FILE)
log.debug('Started logging')

In [16]:
def compute(seq, ref, dis, mask, iteration, length):
    log.debug('Computations started on sequence:')
    log.debug(dis)
        
    time_start_video = time()
    try:
        if not IDLE:
            ssim_vals_simple = []
            ssim_vals_weighted = []
            ssim_vals_fullframe = []
            
            log.debug('Preparing video readers')
            vr_dis = VideoReader(dis, verbose=False)
            vr_ref = VideoReader(ref, shape=vr_dis.shape)
            vr_mask = VideoReader(mask, shape=vr_dis.shape)
            log.debug('Video readers ready')

            for batch_num, (batch_ref, batch_dis, batch_mask) in enumerate(zip(vr_dis, vr_ref, vr_mask)):
                total_batches = vr_dis.len // vr_dis.batch_size
                log.debug('Batch progress: %3d%% [%03d/%03d]' % (batch_num / total_batches * 100, batch_num, total_batches))
                log.debug(f'Shapes: {batch_ref.shape}, {batch_dis.shape}, {batch_mask.shape}')
                log.debug(f'Means:, {batch_ref.mean()}, {batch_dis.mean()}, {batch_mask.mean()}')
                    
                ssim_vals_simple.append(
                    piq.ssim(batch_ref, batch_dis))
                ssim_vals_weighted.append(
                    sal_ssim.ssim(batch_ref, batch_dis, batch_mask))
                ssim_vals_fullframe.append(
                    sal_ssim_fullframe.ssim(batch_ref, batch_dis, batch_mask))
                log.debug(f'Ssim:, {ssim_vals_simple[-1]}, {ssim_vals_weighted[-1]}, {ssim_vals_fullframe[-1]}')
            
            dis_filename = dis.split('/')[-1]
            with open(str(PKL_ARRAYS/ f'{dis_filename}_simple.pkl'), 'wb') as f:
                log.debug(f'Save dir:, {PKL_ARRAYS}')
                log.debug(f'Saved to:, {str(PKL_ARRAYS/ f"{dis_filename}_simple.pkl")}')
                pkl.dump(ssim_vals_simple, f)
            with open(str(PKL_ARRAYS / f'{dis_filename}_weighted.pkl'), 'wb') as f:
                pkl.dump(ssim_vals_weighted, f)
            with open(str(PKL_ARRAYS / f'{dis_filename}_fullframe.pkl'), 'wb') as f:
                pkl.dump(ssim_vals_fullframe, f)
                    
            metric_simple = safe_mean(ssim_vals_simple)
            metric_weighted = safe_mean(ssim_vals_weighted)
            metric_fullframe = safe_mean(ssim_vals_fullframe)
        else:
            metric_simple = 0.1
            metric_weighted = 0.2
            metric_fullframe = 0.3
                      
        time_calc = int(time() - time_start_video)
                      
                    
        row_1 = [seq, ref, dis, 'ssim_simple', round(metric_simple, 4), False, time_calc / 3]
        row_2 = [seq, ref, dis, 'ssim_weighted', round(metric_weighted, 4), True, time_calc / 3]
        row_3 = [seq, ref, dis, 'ssim_fullframe', round(metric_fullframe, 4), True, time_calc / 3]
                      
        writer_results.write_row(row_1)
        writer_results.write_row(row_2)
        writer_results.write_row(row_3)
                      
        log.info("%3d%% [%03d/%03d]      %.3f  %.3f  %.3f      %02d:%02d      %-70s " % 
              (iteration / length * 100, iteration, length, 
               metric_simple, metric_weighted, metric_fullframe,
               time_calc // 60, time_calc % 60, dis_filename))
    except:
        print(traceback.format_exc())
        log.error(f'seq: {seq}, ref: {ref}, dis: {dis}, mask: {mask}')

In [5]:
def make_runconfig(dataset):
    runconfig = deepcopy(dataset)
    for seq in dataset:
        del runconfig[seq]['dis']
        runconfig[seq]['exp'] = []
        runconfig[seq]['exp'] = [
            {
                'dis': dis,
                # 'metric': metric,
                # 'mask_mode': mask_mode
            } for
            # metric in metrics for mask_mode in mask_modes for 
            dis in dataset[seq]['dis']
        ]
    return runconfig

In [6]:
dataset = json_representation_of_dataset(DATASET)
global_runconfig = make_runconfig(dataset)

In [7]:
print(json.dumps(global_runconfig, indent=4)[:1000])

{
    "crowd_run": {
        "mask": "/home/a_mos/2019/masks/crowd_run_mask.mp4",
        "ref": "/home/a_mos/2019/ref/crowd_run_short_1920x1080_50.yuv",
        "exp": [
            {
                "dis": "/home/a_mos/2019/seq/crowd_run_x265/enc_res_bytedance_mv_subjective_crowd_run_short_1000.mp4"
            },
            {
                "dis": "/home/a_mos/2019/seq/crowd_run_x265/enc_res_bytedance_mv_subjective_crowd_run_short_2000.mp4"
            },
            {
                "dis": "/home/a_mos/2019/seq/crowd_run_x265/enc_res_bytedance_mv_subjective_crowd_run_short_4000.mp4"
            },
            {
                "dis": "/home/a_mos/2019/seq/crowd_run_x265/enc_res_huawei_mv_subjective_crowd_run_short_1000.mp4"
            },
            {
                "dis": "/home/a_mos/2019/seq/crowd_run_x265/enc_res_huawei_mv_subjective_crowd_run_short_2000.mp4"
            },
            {
                "dis": "/home/a_mos/2019/seq/crowd_run_x265/enc_res_huawei_mv_subjecti

In [8]:
writer_results = MyWriter(RESULTS)
writer_results.write_row(HEADERS)

all_sequences = np.array(['crowd_run', 'kayak_trip', 'tractor', 'making_alcohol', 'wedding_party'])
sequences = all_sequences[[0]]
runconfig = {seq: global_runconfig[seq] for seq in sequences}
length = sum(len(runconfig[seq]['exp']) for seq in runconfig)
print(length)

36


In [9]:
# print(json.dumps(runconfig, indent=4)[:])

In [ ]:
IDLE = False

command_handler = CommandHandler(2)

iteration = 0
for seq in runconfig:
    for exp in runconfig[seq]['exp']:
        ref = runconfig[seq]['ref']
        mask = runconfig[seq]['mask']
        dis = exp['dis']

        args = (seq, ref, dis, mask, iteration, length)
        command_handler.run(target=compute, args=args)

        iteration += 1